# ZaloAI Lyrics Alignment Predict

In [1]:
import os
import json
from time import perf_counter

from tqdm import tqdm
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from datasets import Dataset, Audio

from predict_utils import end_to_end_align, load_single_example
from segment_utils import load_annotate_lyrics, do_segmentation_transform

cache_dir = './cache/'
processor = Wav2Vec2Processor.from_pretrained("nguyenvulebinh/wav2vec2-base-vietnamese-250h", cache_dir=cache_dir)
model = Wav2Vec2ForCTC.from_pretrained("hungngocphat01/Checkpoint_zaloAI_11_19_2022", cache_dir=cache_dir)

SONGS_PATH = os.environ["SONGS_PATH"]
LYRICS_PATH = os.environ["LYRICS_PATH"]
SUBMISSION_PATH = os.environ["SUBMISSION_PATH"]

file_ids = os.listdir(SONGS_PATH)
file_ids = [file.split(".")[0] for file in file_ids]

/home/hiraki/mambaforge/envs/zaloai/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
def alignment_song(file):
    song = load_single_example(file)
    alignment = end_to_end_align(song, processor, model)

    return {
        "song": file,
        "alignment": alignment
    }

In [7]:
start_time = perf_counter()

In [8]:
align_results = []
for file_id in tqdm(file_ids):
    align_results.append(alignment_song(file_id))

  0%|          | 0/264 [00:00<?, ?it/s]/home/hiraki/mambaforge/envs/zaloai/lib/python3.8/site-packages/transformers/models/wav2vec2/processing_wav2vec2.py:155: UserWarning: `as_target_processor` is deprecated and will be removed in v5 of Transformers. You can process your labels by using the argument `text` of the regular `__call__` method (either in the same call as your audio inputs, or in a separate call.
  warnings.warn(
100%|██████████| 264/264 [22:40<00:00,  5.15s/it]


In [9]:
# Do sentence segmentation
annotate_lyrics = load_annotate_lyrics(file_ids)
s = do_segmentation_transform(align_results, annotate_lyrics)

Noise songs 1


In [10]:
if not os.path.exists(SUBMISSION_PATH):
    os.mkdir(SUBMISSION_PATH)

for key, data in s.items():
    with open(os.path.join(SUBMISSION_PATH, key + ".json"), "wt") as f:
        json.dump(data, f, ensure_ascii=False)

In [11]:
end_time = perf_counter()

In [12]:
elapsed_inference_time = end_time - start_time

In [13]:
elapsed_inference_time

1360.861822103001